In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
import imageio as io
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### The Network
This is the convolutional network that will learn to classify eruptions from the IR pictures of the lava lake on Mt. Erebus. It is written in Pytorch and modeled after the VGG net. It consist of 6 convolutional layers with a max pooling layer after every other conv. layer. These layers pick out features in the pictures used to classify the images. After that there are 3 linear, fully connected layers used to actually classify the images. Totalling 9 layers in all.

In [2]:
class piccnn(nn.Module):
    def __init__(self):
        super(piccnn, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 34, 3)
        self.conv2 = nn.Conv2d(34, 34, 3)
        self.conv3 = nn.Conv2d(34, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.conv5 = nn.Conv2d(64, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)

        self.pool = nn.MaxPool2d(3, 3)

        self.fc1 = nn.Linear(40320, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))
        x = x.view(-1, self.numflatfeatures(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def numflatfeatures(self, x):
        size = x.size()[1:]  # all dimensions except the batch
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

## Dataset class
This is a class that will contain all the information necessary to manipulate the data or images. It allows us to get the total size of the dataset and pull small batches of images with labels for training or validation.

In [3]:
class Dataset:
    def __init__(self, epaths, npaths, data_folder):
        '''
        This is the dataset class; it holds file paths for the data and retrieves the photos as they 
        are needed, outputting them as numpy arrays to be used in training run
        epaths: {numpy array}; array of file names for events in data folder
        npaths: {numpy array}; array of file names for nonevents in data folder
        data_folder: {String}; dataset directory or folder
        '''
        self.epaths = epaths
        self.npaths = npaths
        self.folder = data_folder
        
    def getSize(self):
        '''
        Returns the total size of the dataset
        '''
        return len(self.epaths) + len(self.npaths)
        
    def getRandomBatchTrain(self, batch_size):
        '''
        Returns a randomized batch of data and its labels are structured for BCEWithLogitsLoss
        batch_size: {Integer}; this is the number of photos to pull from the dataset
        '''
        np.random.shuffle(self.epaths)
        np.random.shuffle(self.npaths)
        batch = list()
        labels = list()
        it = int(batch_size / 2)
        for i in range(it):
            image1 = io.imread(os.path.join(self.folder, self.epaths[i]))
            image2 = io.imread(os.path.join(self.folder, self.npaths[i]))
            batch.append(image1)
            labels.append([0, 1])
            batch.append(image2)
            labels.append([1, 0])
            
        if len(batch) > batch_size:
            batch = batch[:batch_size]
            
        return np.array(batch), np.array(labels)
    
    def getRandomBatchVal(self, batch_size):
        '''
        Returns a randomized batch of data and its labels
        batch_size: {Integer}; this is the number of photos to pull from the dataset
        '''
        np.random.shuffle(self.epaths)
        np.random.shuffle(self.npaths)
        batch = list()
        labels = list()
        it = int(batch_size / 2)
        for i in range(it):
            image1 = io.imread(os.path.join(self.folder, self.epaths[i]))
            image2 = io.imread(os.path.join(self.folder, self.npaths[i]))
            batch.append(image1)
            labels.append(1)
            batch.append(image2)
            labels.append(0)
            
        if len(batch) > batch_size:
            batch = batch[:batch_size]
            
        return np.array(batch), np.array(labels)

## Training Run
This function is used to train the network. The function loops over the data for the specified number of times, pulling a minibatch of specified size, norms the images, feeds it into the network, then depending on performance updates the networks weights. It provides updates on the training by outputting the total loss after each epoch. After the training is finished it gives the total time taken for the function to run.

In [4]:
def training_run(network, dataset, criterion, optimizer, batch_size,
                 learning_rate, scheduler, num_epochs=200):
    '''
    This is the function that will train the network.
    network: {pytorch network}; the piccnn above
    criterion: {pytorch criterion fucntion}; this is the loss function
    optimizer: {pytorch optimizer function}; how network is updated; I use standard gradient descent (most common)
    batch_size: {Integer}; number of pics to pull for training batch usually set to 125/250; need to create dataset first
    learning_rate: {Float}; how quickly the network is trained
    scheduler: {pytorch scheduler function}; updates learning rate if the network stalls in its learning (shouldn't have to mess with this)
    num_epochs: {Integer}; number of times the dataset it run through.
    '''
    timeit = time.time()
    norm = nn.BatchNorm2d(1)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-'*10)
        
        running_loss = 0.0
        
        num_batches = int(dataset.getSize() / batch_size) - 1
        
        for i in range(num_batches):
            data, labels = dataset.getRandomBatchTrain(batch_size)
            data = torch.from_numpy(data).float()
            labels = torch.tensor(labels)
#             data = norm(data.view(batch_size, 1, 480, 640))
            data = norm(torch.unsqueeze(data, 1))
            data = Variable(data).float().to(device)
            labels = Variable(labels).float().to(device)
            
            # Start of Training
            optimizer.zero_grad()
            output = network(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            # Update the weights
            for f in network.parameters():
                f.data -= (f.grad.data * learning_rate)
                
            running_loss += loss.item() * data.size(0)
            
        epoch_loss = running_loss / dataset.getSize()
        scheduler.step(epoch_loss)
        
        print('Loss: {:.4f}'.format(epoch_loss))
        print()
        
    total_time = time.time() - timeit
    print('Training done in {:.0f}m {:.0f}s'.format(total_time // 60, total_time % 60))
    
    return network

## Validation Function
This does much the same thing as the training_run function. The difference is that it does not update the network and it also keeps track of the accuracy of the network on the new dataset. This accuracy is then output after the validation data has been looped through along with the total run time of the function.

In [6]:
def validation_run(net, dataset, batch_size=10):
    timeit = time.time()
    norm = nn.BatchNorm2d(1)
    correct = 0
    wrong = 0
        
    num_batches = int(dataset.getSize() / batch_size) - 1
        
    for i in range(num_batches):
        data, labels = dataset.getRandomBatchVal(batch_size)
        data = torch.from_numpy(data).float()
        data = norm(torch.unsqueeze(data, 1))
        data = Variable(data).float().to(device)
            
        output = net(data)
        _, preds = torch.max(output, 1)
        preds = preds.data.cpu().numpy()
        for i in range(batch_size):
            if preds[i] == labels[i]:
                correct += 1
            else:
                wrong += 1
                
    grade = correct / (correct+wrong)
    print('Accuracy: ', grade*100)
    print('Correct: ', correct)
    print('Missed: ', wrong)
    print()
    total_time = time.time() - timeit
    print('Training done in {:.0f}m {:.0f}s'.format(total_time // 60, total_time % 60))

## Prep for Training
Here the training dataset object is created, the network is initalized, and the functions and variables used in the training run are created. The optimizer is the Standard Gradient Descent, the scheduler simply reduces the learning_rate when the network's loss after each epoch reaches a plateau, and the criterion is BCEwithLogitsLoss as explained at https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss. 

In [7]:
home_dir = 'dataset'
events = np.genfromtxt('event_files.csv', delimiter=',', dtype=str)
nonevents = np.genfromtxt('nonevents.csv', delimiter=',', dtype=str)

dataset = Dataset(events, nonevents, home_dir)
net = piccnn()
net.to(device)

learning_rate = 0.001
batchSize = 15

lenneg = len(nonevents)
lenpos = len(events)

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.1, verbose=True)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([lenneg/(lenpos+lenneg), lenpos/(lenpos+lenneg)]).to(device))

The function training_run is called and the network is trained on the dataset intialized above. Progress is output after each epoch. 

In [8]:
net = training_run(net, dataset, criterion, optimizer, batchSize, learning_rate, scheduler, num_epochs=75)

Epoch 1/55
----------
Loss: 0.4227

Epoch 2/55
----------
Loss: 0.4214

Epoch 3/55
----------
Loss: 0.4199

Epoch 4/55
----------
Loss: 0.4183

Epoch 5/55
----------
Loss: 0.4168

Epoch 6/55
----------
Loss: 0.4153

Epoch 7/55
----------
Loss: 0.4139

Epoch 8/55
----------
Loss: 0.4125

Epoch 9/55
----------
Loss: 0.4112

Epoch 10/55
----------
Loss: 0.4098

Epoch 11/55
----------
Loss: 0.4085

Epoch 12/55
----------
Loss: 0.4073

Epoch 13/55
----------
Loss: 0.4060

Epoch 14/55
----------
Loss: 0.4048

Epoch 15/55
----------
Loss: 0.4036

Epoch 16/55
----------
Loss: 0.4024

Epoch 17/55
----------
Loss: 0.4012

Epoch 18/55
----------
Loss: 0.4001

Epoch 19/55
----------
Loss: 0.3990

Epoch 20/55
----------
Loss: 0.3979

Epoch 21/55
----------
Loss: 0.3968

Epoch 22/55
----------
Loss: 0.3957

Epoch 23/55
----------
Loss: 0.3946

Epoch 24/55
----------
Loss: 0.3933

Epoch 25/55
----------
Loss: 0.3923

Epoch 26/55
----------
Loss: 0.3910

Epoch 27/55
----------
Loss: 0.3901

Epoch 28/5

## Prep for Validation
The validation dataset it initalized. This is a completely new dataset where the network has not seen any of the images it contains. This new dataset is used to make sure the network actually learned some. After the validation dataset is loaded the network is put into evaluation mode and we are ready to test if it learned anything.

In [9]:
home_dir = 'validation_dataset'
event_paths = np.genfromtxt('validation_events.csv', delimiter=',', dtype=str)
nonevent_paths = np.genfromtxt('validation_nonevents.csv', delimiter=',', dtype=str)

val_data = Dataset_Validation(event_paths, nonevent_paths, home_dir)

net.eval()

piccnn(
  (conv1): Conv2d(1, 34, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(34, 34, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(34, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=40320, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=2, bias=True)
)

The function validation_run is called and the networks ability to detect eruptions based on the IR images is put to the test. At the end the accuracy as well as the number of correctly and incorrectly labeled images is printed out along with the total time for the function to run.

In [10]:
validation_run(net, val_data)

Accuracy:  0.9514285714285714
Correct:  333
Missed:  17
